# Chapter 05 이미지 처리 능력이 탁월한 CNN

## 5.2 CNN 모델 구현하기

In [1]:
# 모듈 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
# gpu 사용가능 여부 확인
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
USE_CUDA

True

In [4]:
# 하이퍼파라미터 설정
EPOCHS = 40
BATCH_SIZE = 64

In [7]:
# 데이터로더 정의
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)

현재 만드는 CNN의 커널은 5x5, 컨볼루션 계층은 2개

[Conv2d 층 설명](https://gaussian37.github.io/dl-pytorch-conv2d/)

In [22]:
# CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.drop = nn.Dropout2d()

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))

        x = x.view(-1, 320)

        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return x

In [23]:
# model과 최적화함수
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [24]:
# 훈련 함수
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} ({100.*batch_idx/len(train_loader):.0f}%)]\tLoss:{loss.item():.6f}')

In [29]:
len(train_loader), len(train_loader.dataset)

(938, 60000)

In [26]:
# 평가 함수
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

모델 실행

In [27]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

Train Epoch: 1 [0/60000 (0%)]	Loss:2.313305
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.370155
Train Epoch: 1 [25600/60000 (43%)]	Loss:1.088256
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.860652
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.747864
[1] Test Loss: 0.6642, Accuracy: 74.47%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.963815
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.785075
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.856021
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.665301
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.767851
[2] Test Loss: 0.5618, Accuracy: 78.50%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.534215
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.672763
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.747651
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.554629
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.652202
[3] Test Loss: 0.5137, Accuracy: 80.29%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.810447
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.802413
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.581880
Train Epoch: 4 [38

### 전체코드 및 연습

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [49]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [50]:
EPOCHS = 50  # Horizon 추가해서  10에폭 늘림
BATCH_SIZE = 64

In [51]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),        # 추가함. 데이터가 늘어남에 따라 에폭 수 늘려야될것으로 보임
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE, shuffle=True
)

In [57]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return x

In [58]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [62]:
def train(model, train_loader, optimizer, epoch):
    print(f"Train Epoch : {epoch}")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        # if batch_idx % 200 == 0:
        #     print(f"[{batch_idx*len(data)}/{len(train_loader.dataset)} ({100.*batch_idx/len(train_loader):.0f}%)]\tLoss: {loss.item:.6f}")

In [63]:
def evaluate(mode, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [64]:
from tqdm import tqdm

for epoch in tqdm(range(1, EPOCHS + 1)):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f"[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

  0%|          | 0/50 [00:00<?, ?it/s]

Train Epoch : 1


  2%|▏         | 1/50 [00:22<18:07, 22.20s/it]

[1] Test Loss: 0.6868, Accuracy: 74.17%
Train Epoch : 2


  4%|▍         | 2/50 [00:43<17:34, 21.96s/it]

...

[48] Test Loss: 0.3310, Accuracy: 87.61%
Train Epoch : 49


 98%|█████████▊| 49/50 [18:01<00:22, 22.07s/it]

[49] Test Loss: 0.3311, Accuracy: 87.59%
Train Epoch : 50


100%|██████████| 50/50 [18:23<00:00, 22.06s/it]

[50] Test Loss: 0.3305, Accuracy: 87.59%


성능이 더 안좋아짐